<a href="https://colab.research.google.com/github/brandon-setegn/data-mining-business-analytics/blob/master/west-roxbury-home-values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>